Import all necessary libraries and install everything you need for training:

In [1]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score, recall_score
import torch
from numba import cuda
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
#!pip install -q transformers

# Install the simpletransformers
#!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

In [3]:
# Install wandb
#!pip install -q wandb

import wandb

# Login to wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tajak (use `wandb login --relogin` to force relogin)


True

In [4]:
# Clean the GPU cache

cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()


### Import the data

In [5]:
# CORE-main (texts annotated with main CORE labels)
train_df = pd.read_csv("data-splits/CORE-main-subset-train.csv", index_col = 0)
dev_df = pd.read_csv("data-splits/CORE-main-subset-dev.csv", index_col = 0)
test_df = pd.read_csv("data-splits/CORE-main-subset-test.csv", index_col = 0)

print("CORE-main train shape: {}, Dev shape: {}, Test shape: {}.".format(train_df.shape, dev_df.shape, test_df.shape))

CORE-main train shape: (10256, 2), Dev shape: (3419, 2), Test shape: (3419, 2).


In [6]:
train_df.head()

,text,labels
38864,I have never been able to ejaculate inside any...,Interactive Discussion
22351,Testimonials We have had the privilege of find...,Interactive Discussion
24938,"Saturday, November 03, 2012 Every time I check...",Narrative
39572,Why Didn't Nokia Use Android? Because That Wou...,Narrative
4574,A veterinarian specialising in wild horse immu...,Narrative


## Training and saving

We will use the multilingual XLM-RoBERTa model
https://huggingface.co/xlm-roberta-base

In [7]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [8]:
# Create a list of labels
LABELS = train_df.labels.unique().tolist()
LABELS

['Interactive Discussion',
 'Narrative',
 'Informational Description/Explanation',
 'Lyrical',
 'Opinion',
 'Informational Persuasion',
 'Spoken',
 'How-To/Instructional',
 'Other']

In [9]:
# Initialize Wandb
wandb.init(project="CORE-hyperparameter-search", name="training-the-model")

In [10]:
# Calculate how many steps will each epoch have
# Num steps in epoch = training samples / batch size
steps_per_epoch = int(10256/8)
steps_per_epoch

1282

I evaluated per every 5th epoch - per 6410 steps. I first trained the model while evaluating it to find the optimal number of epochs. As previous experiments on smaller datasets showed that already 10-20 epochs are enough, we will here try up to 20 epochs, since this dataset is much bigger.

In [14]:
# Create a TransformerModel and evaluate during training
epoch = 20

roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": epoch,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            # Use these parameters if you want to evaluate during training
            "evaluate_during_training": True,
            "evaluate_during_training_steps": steps_per_epoch*5,
            "evaluate_during_training_verbose": True,
            "use_cached_eval_features": True,
            'reprocess_input_data': True,
            "labels_list": LABELS,
            # The following parameters (no_cache, no_save) are commented out if I want to save the model
            "no_cache": True,
            # Disable no_save: True if you want to save the model
            "no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'CORE-hyperparameter-search',
            "silent": True,
            }
        )

# Train the model and evaluate during training
roberta_base_model.train_model(train_df, eval_df = dev_df)

Evaluation during training showed that the number of epochs before the eval_loss starts rising is somewhere between epochs 2 and 6. We then trained the model for epochs 2, 4, 6 and 8 to find the optimum number.

In [11]:
# Create a file to save results into (you can find it under Data: Output). Be careful, run this step only once to not overwrite the results file.
results = []

with open("results/CORE-main-Experiments-Results.json", "w") as results_file:
    json.dump(results,results_file, indent= "")

In [12]:
# Open the main results file:

previous_results_file = open("results/CORE-main-Experiments-Results.json")
previous_results = json.load(previous_results_file)
len(previous_results)

0

In [13]:
def testing(test_df, test_name, epoch):
    """
    This function takes the test dataset and applies the trained model on it to infer predictions.
    It also prints and saves a confusion matrix, calculates the F1 scores and saves the results in a list of results.

    Args:
    - test_df (pandas DataFrame)
    - test_name
    - epoch: num_train_epochs
    """
    # Get the true labels
    y_true = test_df.labels

    model = roberta_base_model
    
    # Calculate the model's predictions on test
    def make_prediction(input_string):
        return model.predict([input_string])[0][0]

    y_pred = test_df.text.apply(make_prediction)

    # Calculate the scores
    macro = f1_score(y_true, y_pred, labels=LABELS, average="macro")
    micro = f1_score(y_true, y_pred, labels=LABELS,  average="micro")
    print(f"Macro f1: {macro:0.3}, Micro f1: {micro:0.3}")

    # Plot the confusion matrix:
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, cmap="Oranges")
    for (i, j), z in np.ndenumerate(cm):
        plt.text(j, i, '{:d}'.format(z), ha='center', va='center')
    classNames = LABELS
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=90)
    plt.yticks(tick_marks, classNames)
    plt.title(f"{test_name}")

    plt.tight_layout()
    fig1 = plt.gcf()
    plt.show()
    plt.draw()
    fig1.savefig(f"Confusion-matrix-{test_name}.png",dpi=100)

    # Save the results:
    rezdict = {
        "experiment": test_name,
        "num_train_epochs": epoch,
        "train_batch_size":8,
        "learning_rate": 1e-5,
        "microF1": micro,
        "macroF1": macro,
        "y_true": y_true.to_dict(),
        "y_pred": y_pred.to_dict(),
        }
    previous_results.append(rezdict)

    #Save intermediate results (just in case)
    backup = []
    backup.append(rezdict)
    with open(f"backup-results-{test_name}.json", "w") as backup_file:
        json.dump(backup,backup_file, indent= "")

In [15]:
# Train the model for various epochs to find the optimum number
epochs = [2, 4, 6, 8]

for epoch in epochs:
    roberta_base_model = ClassificationModel(
                "xlmroberta", "xlm-roberta-base",
                num_labels=len(LABELS),
                use_cuda=True,
                args= {
                    "overwrite_output_dir": True,
                    "num_train_epochs": epoch,
                    "train_batch_size":8,
                    "learning_rate": 1e-5,
                    "labels_list": LABELS,
                    # The following parameters (no_cache, no_save) are commented out if I want to save the model
                    "no_cache": True,
                    # Disable no_save: True if you want to save the model
                    "no_save": True,
                    "max_seq_length": 512,
                    "save_steps": -1,
                    # Only the trained model will be saved - to prevent filling all of the space
                    "save_model_every_epoch":False,
                    "wandb_project": 'CORE-hyperparameter-search',
                    "silent": True,
                    }
                )

    # Train the model
    roberta_base_model.train_model(train_df)
    
    # Test the model on dev_df
    testing(dev_df, f"Dev-epoch-search:{epoch}", epoch)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Training loss,█▇▆▅▁▆▃▄▂▅▄▄▂▇▆▃▅▂▃▃▃▂
global_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
lr,▁▄████▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄
Training loss,0.83266
global_step,1100
lr,1e-05


KeyboardInterrupt: 

In [ ]:
# Compare the results by creating a dataframe from the previous_results dictionary:
results_df = pd.DataFrame(previous_results)

results_df

The experiments revealed that the optimum number of epochs is 20.

In [ ]:
# Save the file with updated results.
with open("results/CORE-main-Experiments-Results.json", "w") as results_file:
    json.dump(previous_results,results_file, indent= "")

Train the model and save it.

In [ ]:
# Create a TransformerModel
roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": 20,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            "labels_list": LABELS,
            # The following parameters are commented out because I want to save the model
            #"no_cache": True,
            # Disable no_save: True if you want to save the model
            #"no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'GINCO-hyperparameter-search',
            "silent": True,
            }
        )

In [ ]:
# Train the model
roberta_base_model.train_model(train_df)

In [ ]:
# Save the trained model to Wandb
run = wandb.init(project="GINCO-hyperparameter-search", entity="tajak", name="saving-trained-model")
trained_model_artifact = wandb.Artifact("GINCO-full-set-classifier", type="model", description="a model trained on the (Slovene) GINCO dataset with the full set of labels (only the labels with less than 10 instances were excluded. There are 17 labels: 'Promotion of a Product', 'Forum', 'Instruction', 'News/Reporting', 'Opinionated News', 'Other', 'List of Summaries/Excerpts', 'Invitation', 'Opinion/Argumentation', 'Legal/Regulation', 'Information/Explanation', 'Review', 'Promotion of Services', 'Promotion', 'Announcement', 'Call', 'Correspondence'")
trained_model_artifact.add_dir("outputs")
run.log_artifact(trained_model_artifact)